In [41]:
!pip install -qU "langchain[openai]"

In [42]:
import os

# os.environ["OPENAI_API_KEY"] = "sk-proj-xxxx"

## Runtime & State
- Runtime : Agent 실행에 필요한 공유 정보
    - Context : 사용자 Id나 데이터베이스 연결 URL처럼 Agent 실행 시 변하지 않는 정보 -> 보안 및 권한 제어
    - Store : 사용자의 선호, 이름 등 장기 메모리 -> 개인화 제어
    - Stream writer : Agent 내부 작동 로그 등 실시간 이벤트 출력 가능

- State : Agent가 작동하며 만든 결과물 -> 대화 흐름 제어
    - Message : Model이 Runtime을 참조해가며 만든 결과물
    - tool_results : Tool이 만든 정보 모음

# Node-style vs Wrap-style
- Node-style Hook : 특정 실행 지점 (before/after)에서 순차적으로 동작하는 미들웨어
    - 상태 확인, 로깅, 유효성 검사에 적합

- Wrap-style Hook : 실행 흐름 (model call, tool call 등)을 감싸서(intercept) 조작하거나 흐름을 제어할 수 있는 미들웨어
    - 동적 프롬프트 삽입, 흐름 건너뛰기 등 흐름을 제어하고 변형

### node-style

- Middleware에서 Runtime 정보 접근 방법 1
    - 데코레이터 사용 : @before_agent, @before_model, @after_model, @after_agent
    - 입력 파라미터 : state, runtime (필수)

    - @before_agent, @after_agent : 한 번만 실행
    - @before_model, @after_model : Model이 호출될 때마다 실행

In [43]:
#dataclass : Python에서 데이터 중심 클래스 (data container)를 쉽게 만드는 문법
from dataclasses import dataclass

@dataclass
class Context:
    user_name: str

In [44]:
from langchain.agents.middleware import before_model

@before_model()
def log_before_model(state, runtime):
    print(f"State : {state}")
    print(f"Runtime : {runtime}")
    print(f"사용자 이름 : {runtime.context.user_name}")
    return None

In [45]:
from langchain.agents import create_agent
from langchain.agents.middleware import LLMToolEmulator

agent = create_agent(
    model="gpt-5-mini",
    tools=[],
    middleware=[log_before_model],
    context_schema=Context
)

In [46]:
agent.invoke(
    {"messages": [{"role": "user", "content": "제 이름이 뭐죠?"}]},
    context=Context(user_name="jwg")
)

State : {'messages': [HumanMessage(content='제 이름이 뭐죠?', additional_kwargs={}, response_metadata={}, id='a75a932a-06d2-4248-b171-a70f21f45cec')]}
Runtime : Runtime(context=Context(user_name='jwg'), store=None, stream_writer=<function Pregel.stream.<locals>.stream_writer at 0x7fda21bdaf20>, previous=None)
사용자 이름 : jwg


{'messages': [HumanMessage(content='제 이름이 뭐죠?', additional_kwargs={}, response_metadata={}, id='a75a932a-06d2-4248-b171-a70f21f45cec'),
  AIMessage(content='죄송하지만 지금은 귀하의 이름을 알 수 없습니다. 저는 사용자의 개인 정보를 자동으로 조회할 권한이나 접근이 없어요.\n\n이름을 알려주시면 이 대화에서는 그 이름으로 부르겠습니다. 어떻게 불러드릴까요? (예: 성함, 별명, 혹은 그냥 “님”)', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 399, 'prompt_tokens': 12, 'total_tokens': 411, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D3GMl3vHyX3jBZ9axwyqYqIaVmYvu', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c0891-4fcd-7d73-a081-6a73e619934f-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 12, 'output_tokens'

### wrap-style

- Middleware에서 Runtime 정보 접근 방법 2
    - 데코레이터 사용 (wrap-style) : model이나 tool이 호출될 때 실행과 제어를 가로챌 때 사용
        - @wrap_model_call, @wrap_tool_call
    - 입력 파라미터 : request, handler (필수)
        

In [47]:
from langchain.agents.middleware import wrap_model_call
from langchain.messages import HumanMessage, SystemMessage

@wrap_model_call
def inject_user_name(request, handler):
    # 1 요청 가로채기
    print(f"Request : {request}")

    # 2 request.runtime.context.user_name 접근
    user_name = request.runtime.context.user_name

    # 3 시스템 메시지 삽입 (요청 수정)
    if user_name:
        sys_prompt = f"사용자의 이름은 {user_name}입니다."
        request = request.override(system_prompt=sys_prompt)

    # 4 수정된 요청을 다음 단계(LLM 호출)로 넘김
    return handler(request)

In [48]:
from langchain.agents.middleware import after_model

@after_model
def log_after_model(state, runtime) :
    print(f"모델 응답 완료 후 : {state["messages"][-1].content}")
    return None

In [49]:
agent = create_agent(
    model="gpt-5-mini",
    tools=[],
    middleware=[inject_user_name, log_after_model],
    context_schema=Context
)

In [50]:
agent.invoke(
    {"messages": [{"role": "user", "content": "제 이름이 뭐죠?"}]},
    context=Context(user_name="JWG")
)

Request : ModelRequest(model=ChatOpenAI(profile={'max_input_tokens': 272000, 'max_output_tokens': 128000, 'image_inputs': True, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': True, 'tool_calling': True, 'structured_output': True, 'image_url_inputs': True, 'pdf_inputs': True, 'pdf_tool_message': True, 'image_tool_message': True, 'tool_choice': True}, client=<openai.resources.chat.completions.completions.Completions object at 0x7fda23b89f40>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x7fda23b89340>, root_client=<openai.OpenAI object at 0x7fda23b89fa0>, root_async_client=<openai.AsyncOpenAI object at 0x7fda23b8bf80>, model_name='gpt-5-mini', model_kwargs={}, openai_api_key=SecretStr('**********'), stream_usage=True), messages=[HumanMessage(content='제 이름이 뭐죠?', additional_kwargs={}, response_metadata={}, id='b0530d75-d3be-45b7-a01c-4e1cd61cb849')], system

{'messages': [HumanMessage(content='제 이름이 뭐죠?', additional_kwargs={}, response_metadata={}, id='b0530d75-d3be-45b7-a01c-4e1cd61cb849'),
  AIMessage(content='당신의 이름은 JWG님입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 147, 'prompt_tokens': 24, 'total_tokens': 171, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D3GMs3kbaaeSZ4LgZRZlwLsUnmwVH', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c0891-6a80-7d50-8228-f7980ce3bbf4-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 24, 'output_tokens': 147, 'total_tokens': 171, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, '

### Context, Middleware를 사용해 로그인한 사용자는 고급 모델을 지원하는 Agent 서비스 구축

In [51]:
@dataclass
class Context:
    user_name: str
    is_premium: bool = False

In [52]:
from langchain.chat_models import init_chat_model

@wrap_model_call
def dynamic_model_selector(request, handler):
    is_premium = request.runtime.context.is_premium
    user_name = request.runtime.context.user_name

    if is_premium:
        model_name = "gpt-5"
        print(f"{user_name} 사용자는 프리미엄 회원입니다. GPT-5를 호출합니다.")
    else:
        model_name = "gpt-5-nano"
        print(f"{user_name} 사용자는 일반 회원입니다. GPT-5-nano를 호출합니다.")

    new_model = init_chat_model(model_name)
    new_request = request = request.override(model=new_model)

    return handler(new_request)

In [53]:
agent = create_agent(
    model = "gpt-5-nano",
    tools = [],
    middleware = [dynamic_model_selector],
    context_schema = Context
)

In [54]:
agent.invoke(
    {"messages" : [{"role" : "user", "content" : "안녕하세요"}]},
    context = Context(user_name = "jwg", is_premium = True)
)

jwg 사용자는 프리미엄 회원입니다. GPT-5를 호출합니다.


{'messages': [HumanMessage(content='안녕하세요', additional_kwargs={}, response_metadata={}, id='5d310c65-0c70-4150-8a88-f3e3061ede98'),
  AIMessage(content='안녕하세요! 무엇을 도와드릴까요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 84, 'prompt_tokens': 8, 'total_tokens': 92, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 64, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D3GMwzrnxj2NF6CtSHGzf0aPr7iYO', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c0891-7a0c-7090-9bde-e07875b3fd7d-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 8, 'output_tokens': 84, 'total_tokens': 92, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 64

In [55]:
agent.invoke(
    {"messages" : [{"role" : "user", "content" : "안녕하세요"}]},
    context = Context(user_name = "Amy", is_premium = False)
)

Amy 사용자는 일반 회원입니다. GPT-5-nano를 호출합니다.


{'messages': [HumanMessage(content='안녕하세요', additional_kwargs={}, response_metadata={}, id='9556f873-aeb7-46d0-bd81-c6dc1302634b'),
  AIMessage(content='안녕하세요! 무엇을 도와드릴까요? 궁금한 점이나 필요한 주제가 있으면 말씀해 주세요. 예시로는 번역, 요약, 코드 문제, 학습 자료, 여행 계획, 요리 레시피, 정보 검색 등이 있어요. 어떤 주제로 시작할까요? 필요하시면 간단한 대화 연습도 도와드릴게요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 347, 'prompt_tokens': 8, 'total_tokens': 355, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D3GMzkUf756BmlyEaNpxQ7XPDiU9b', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c0891-854f-79f0-aa8b-0c8a291ed0f5-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 8, 'output_tokens'

## Custom middleware

### node-style

In [56]:
from langchain.agents.middleware import before_agent
from langchain.messages import AIMessage

@before_agent(can_jump_to = ["end"])
def validate_input(state, runtime):
    human_message = state["messages"][-1]
    if "암구호" in human_message.content:
        print("암구호 감지")
        return {
            "messages" : [AIMessage(content = "현재 암구호가 포함되어 있습니다. 보안에 주의하세요.")],
            "jump_to" : "end" # 모델 호출 중단 후 에이전트 종료
        }
    return None

In [57]:
agent = create_agent(
    model="gpt-5-mini",
    middleware=[validate_input],
)

In [58]:
agent.invoke(
    {"messages": [{"role": "user", "content": "오늘의 암구호는 삼각대-자동차 입니다."}]},
)

암구호 감지


{'messages': [HumanMessage(content='오늘의 암구호는 삼각대-자동차 입니다.', additional_kwargs={}, response_metadata={}, id='29202f85-7496-49b4-a1ee-f83394020a22'),
  AIMessage(content='현재 암구호가 포함되어 있습니다. 보안에 주의하세요.', additional_kwargs={}, response_metadata={}, id='c91b3ab7-90d5-4926-bf0a-753964f80f55', tool_calls=[], invalid_tool_calls=[])]}

In [59]:
res = agent.invoke(
    {"messages": [{"role": "user", "content": "오늘의 암구호는 삼각대-자동차 입니다."}]},
)

res["messages"][-1].content

암구호 감지


'현재 암구호가 포함되어 있습니다. 보안에 주의하세요.'

### wrap-style

In [60]:
from langchain_openai import ChatOpenAI

@wrap_model_call
def dynamic_model_selector(request, handler):
    # 최근 사용자의 입력 메시지 추출
    last_msg = request.messages[-1].content if request.messages else ""
    msg_len = len(last_msg)

    # 길이에 따라 모델 선택
    if msg_len < 10:
        model_name = "gpt-5-nano"
    elif msg_len < 30:
        model_name = "gpt-5-mini"
    else:
        model_name = "gpt-5"

    print(f"메시지 길이: {msg_len}, 선택된 모델: {model_name}")

    # request.model을 새로운 모델로 교체
    new_model = init_chat_model(model_name)
    new_request = request.override(model = new_model)

    # 수정된 요청으로 LLM 호출
    return handler(new_request)

In [61]:
agent = create_agent(
    model="gpt-5-mini",
    tools=[],
    middleware=[dynamic_model_selector],
)

In [62]:
agent.invoke(
    {"messages": [{"role": "user", "content": "안녕하세요"}]},
)

메시지 길이: 5, 선택된 모델: gpt-5-nano


{'messages': [HumanMessage(content='안녕하세요', additional_kwargs={}, response_metadata={}, id='92df5dc9-72a6-4e8a-8f6c-ae2ac57dfab0'),
  AIMessage(content='안녕하세요! 무엇을 도와드릴까요? 필요하신 것이 있으면 말씀해 주세요. 예를 들어 한국어 공부 도와주기, 번역, 정보 검색, 코드 도움, 글쓰기 첨삭, 문제 풀이 등 어떤 주제든 좋습니다. 특정 주제나 질문이 있나요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 332, 'prompt_tokens': 8, 'total_tokens': 340, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D3GNArwmnVmgwDApB4RHERPzsanCq', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c0891-a942-75a2-a85f-323be0f92257-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 8, 'output_tokens': 332, 'total_tokens'

In [63]:
agent.invoke(
    {"messages": [{"role": "user", "content": "안녕하세요. 저는 jwg입니다."}]},
)

메시지 길이: 17, 선택된 모델: gpt-5-mini


{'messages': [HumanMessage(content='안녕하세요. 저는 jwg입니다.', additional_kwargs={}, response_metadata={}, id='261639fa-0ea8-4831-bb5e-84442289963b'),
  AIMessage(content='안녕하세요 jwg님, 반갑습니다! 무엇을 도와드릴까요? 원하시는 언어(한국어/English)나 주제가 있으면 알려 주세요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 174, 'prompt_tokens': 14, 'total_tokens': 188, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-mini-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D3GNGEc5h6USJ85p2Lfau7UyFniFs', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c0891-ca9d-7841-ac94-04382f28b790-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 14, 'output_tokens': 174, 'total_tokens': 188, 'input_token_details': {'audio':

In [64]:
agent.invoke(
    {"messages": [{"role": "user", "content": "안녕하세요. 저는 jwg입니다. 인공지능에 관해 궁금한게 있어요."}]},
)

메시지 길이: 36, 선택된 모델: gpt-5


{'messages': [HumanMessage(content='안녕하세요. 저는 jwg입니다. 인공지능에 관해 궁금한게 있어요.', additional_kwargs={}, response_metadata={}, id='1bdcdada-c66b-4b80-ada8-49e446cc550f'),
  AIMessage(content='안녕하세요, jwg님! 반갑습니다. 인공지능 어떤 점이 궁금하신가요? 알고 싶은 범위를 알려주시면 수준에 맞춰 설명드릴게요(초보/실무/연구).\n\n예를 들어 이런 주제들이 있어요:\n- 인공지능/머신러닝/딥러닝의 차이와 기본 원리\n- 대규모 언어모델(LLM)이 어떻게 동작하는지\n- 학습 데이터, 파인튜닝, RAG(검색결합) 개념\n- 프롬프트 엔지니어링 요령\n- 모델 선택과 배포(오픈소스 vs 상용)\n- 윤리·편향·개인정보 이슈\n- 실제 적용 사례(문서 요약, 챗봇, 추천, 컴퓨터비전 등)\n- 시작을 위한 로드맵(필수 수학/파이썬/프레임워크)\n\n원하시는 주제를 말해주시면 바로 설명드리겠습니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 488, 'prompt_tokens': 27, 'total_tokens': 515, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 256, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D3GNJm3G0hBBpqa

## Guardrails
- 책임있는, 신뢰할 수 있는 AI 애플리케이션 구축에 대한 관심 증가

- 랭체인 지원 가드레일
    - 구현 방식
        - 결정론적(deterministic) 가드레일 : 정규식, 키워드 매칭을 활용한 가드레일
            - 장점 : 비용이 저렴함, 빠름
            - 단점 : 미묘한 부분을 놓칠 수 있음
            - 대표 : PII 커스텀 탐지기
        
        - 모델 기반 가드레일 : LLM 모델의 평가 기반 탐지기
            - 장점 : 미세한 문제까지 포착 가능
            - 단점 : 비용 및 시간 소요
            - 대표 : human-in-the-loop

- Custom Guardrails (커스텀 가드레일)
    - 구현 방식
        - Before Agent Guardrails
        - After Agent Guardrails
        - Combine Multiple Guardrails (Before + After)

#### Before agent Guardrails
- 차단할 키워드 사전 정의
- 차단 키워드 감지 시 바로 return
- Middleware 탑재한 Agent 생성

In [65]:
# 차단 및 제재 키워드 정의
forbidden_topics = {
    "cheating": ["답지", "정답 알려줘", "숙제 대신", "써줘", "베끼기"], # 부정행위 관련
    "distraction": ["롤", "게임", "유튜브", "아이돌", "웹툰"],         # 학습 방해 요소
    "harmful": ["담배", "술", "폭력", "싸움", "바보"]                 # 유해 콘텐츠
}

In [66]:
from langchain.agents.middleware import before_agent

@before_agent(can_jump_to=["end"])
def education_guardrail(state, runtime):
    """
    학생의 질문 의도를 파악하여 교육적이지 않거나 부정행위가 의심될 경우,
    LLM(AI)에게 질문을 넘기지 않고 교육적인 멘트로 즉시 교정합니다.
    """

    # 1. 메시지 유효성 검사
    if not state["messages"]:
        return None

    last_message = state["messages"][-1]
    if last_message.type != "human":
        return None

    user_text = last_message.content

    # 2. 카테고리별 검사 로직
    # 단순히 막는 것을 넘어, '왜' 안되는지 카테고리별로 다른 피드백 주기

    # Case A: 부정행위 방지 (Cheating Prevention)
    # AI가 숙제를 통째로 해주는 것을 방지
    for keyword in forbidden_topics["cheating"]:
        if keyword in user_text:
            return {
                "messages": [{
                    "role": "assistant",
                    "content": "🚫 스스로 고민해봐야 실력이 늘어요! 정답을 바로 알려드리는 대신, 힌트를 드릴까요? 어떤 부분이 가장 어려운지 말해주세요."
                }],
                "jump_to": "end"
            }

    # Case B: 학습 집중 유도 (Focus Management)
    # 공부 중에 게임이나 딴짓 이야기를 하면 다시 공부로 유도
    for keyword in forbidden_topics["distraction"]:
        if keyword in user_text:
            return {
                "messages": [{
                    "role": "assistant",
                    "content": "⏰ 지금은 공부에 집중할 시간이에요! 딴짓은 쉬는 시간에 하고, 지금 풀고 있는 문제에 집중해볼까요?"
                }],
                "jump_to": "end"
            }

    # Case C: 유해 콘텐츠 차단 (Safety)
    # 교육 서비스의 브랜드 안전성(Brand Safety)을 위한 기능
    for keyword in forbidden_topics["harmful"]:
        if keyword in user_text:
            return {
                "messages": [{
                    "role": "assistant",
                    "content": "⚠️ 부적절한 대화 주제입니다. 바르고 고운 말을 사용해주세요."
                }],
                "jump_to": "end"
            }

    # 3. 통과 (Pass)
    # 위 조건들에 걸리지 않으면 정상적으로 AI 튜터(LLM)가 답변 생성
    return None

In [67]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    tools=[],
    middleware=[education_guardrail],
)

In [69]:
agent.invoke({
    "messages": [{"role": "user", "content": "피타고라스의 정리가 이해가 안 돼. 설명해줘."}]
})

{'messages': [HumanMessage(content='피타고라스의 정리가 이해가 안 돼. 설명해줘.', additional_kwargs={}, response_metadata={}, id='94f0da71-1b05-4a8c-a4d4-915e9b4204c7'),
  AIMessage(content='좋아요! 피타고라스의 정리는 직각삼각형에 특별히 맞는 아주 기본적인 관계예요.\n\n1) 무엇이 성립하나요\n- 직각삼각형에서 두 직각변의 길이를 a와 b, 빗변의 길이를 c라고 하면\n  a^2 + b^2 = c^2\n- 즉 빗변의 제곱은 나머지 두 변의 제곱의 합과 같아요.\n\n2) 왜 그렇게 느껴지나요? 직관적인 그림\n- 각 변의 길이를 제곱하면 그 길이를 가진 정사각형의 넓이가 됩니다.\n- 피타고라스의 정리는 “직각삼각형의 두 직각변으로 만든 정사각형의 넓이 합이 빗변으로 만든 정사각형의 넓이와 같다”는 뜻이에요.\n- 직관적으로 이해하는 한 가지 방법은 큰 정사각형 안에 네 개의 같은 삼각형을 배치하는 구성입니다. 큰 정사각형의 한 변의 길이를 a+b로 두면 넓이는 (a+b)^2가 됩니다.\n  - 네 개의 삼각형 넓이 합은 4 × (1/2 ab) = 2ab가 되고,\n  - 남은 중앙에 남는 모양은 빗변 c의 정사각형의 넓이와 같다는 사실로 이어집니다.\n  - 따라서 (a+b)^2 = c^2 + 2ab 이고, 전개하면 a^2 + 2ab + b^2 = c^2 + 2ab 이므로 a^2 + b^2 = c^2가 됩니다.\n\n3) 간단한 예시로 확인해 보기\n- 3, 4, 5 삼각형: 3^2 + 4^2 = 9 + 16 = 25 = 5^2. 맞네요.\n- 또 다른 예시로 5, 12, 13도: 5^2 + 12^2 = 25 + 144 = 169 = 13^2.\n\n4) 자주 헷갈리는 점\n- 피타고라스는 직각삼각형에서만 성립합니다. 예를 들어 어떤 삼각형이든 두 변의 제곱의 합이 다른 변의 제곱과 같은지로 판단하려면 반드시 그 삼각형이 직각인지 확인해

In [70]:
agent.invoke({
    "messages": [{"role": "user", "content": "나 독후감 쓰기 귀찮은데 숙제 대신 써줘."}]
})

{'messages': [HumanMessage(content='나 독후감 쓰기 귀찮은데 숙제 대신 써줘.', additional_kwargs={}, response_metadata={}, id='dd20ee19-f37d-4081-a9fd-4963f21d7604'),
  AIMessage(content='🚫 스스로 고민해봐야 실력이 늘어요! 정답을 바로 알려드리는 대신, 힌트를 드릴까요? 어떤 부분이 가장 어려운지 말해주세요.', additional_kwargs={}, response_metadata={}, id='d28f5342-1f37-4bb5-8db6-5e596d4024a4', tool_calls=[], invalid_tool_calls=[])]}

In [71]:
agent.invoke({
    "messages": [{"role": "user", "content": "아 공부하기 싫다. 롤 관련 유튜브 영상이나 찾아줘"}]
})

{'messages': [HumanMessage(content='아 공부하기 싫다. 롤 관련 유튜브 영상이나 찾아줘', additional_kwargs={}, response_metadata={}, id='9b63c96e-cdc0-47d5-9184-d8685425584f'),
  AIMessage(content='⏰ 지금은 공부에 집중할 시간이에요! 딴짓은 쉬는 시간에 하고, 지금 풀고 있는 문제에 집중해볼까요?', additional_kwargs={}, response_metadata={}, id='4295d583-c73c-412a-bd84-83c0fc55d1bb', tool_calls=[], invalid_tool_calls=[])]}

In [72]:
agent.invoke({
    "messages": [{"role": "user", "content": "아 너 왜이렇게 바보같냐..."}]
})

{'messages': [HumanMessage(content='아 너 왜이렇게 바보같냐...', additional_kwargs={}, response_metadata={}, id='7220e01f-dc2a-4e95-9810-dfda9455701f'),
  AIMessage(content='⚠️ 부적절한 대화 주제입니다. 바르고 고운 말을 사용해주세요.', additional_kwargs={}, response_metadata={}, id='870ba504-532b-4be0-ae0d-b0899d77bfd1', tool_calls=[], invalid_tool_calls=[])]}

#### After agent Guardrails
- AI Model이 생성한 답변을 평가하는 모델 정의
- after_agent 데코레이터 활용 미들웨어 작성
- 메세지 유효성 검사
- AI 모델을 통한 검증
- 가드레일이 발동하면 AI Message의 content를 수정해 반환

In [73]:
from langchain.chat_models import init_chat_model

safety_model = init_chat_model("gpt-5-nano")

In [81]:
from langchain.agents.middleware import after_agent

@after_agent
def answer_leakage_guardrail(state, runtime) :
    """
    AI가 답변을 생성한 '직후', 사용자에게 보여주기 전에 내용을 검사.
    만약 AI가 문제의 정답을 직접적으로 말해버렸다면, 이를 감지하고 수정.
    """

    # 1. 메시지 유효성 검사
    if not state["messages"]: return None
    last_message = state["messages"][-1]

    # 마지막 메시지가 AI의 답변이 아니면 검사할 필요 없음
    if not isinstance(last_message, AIMessage):
        return None

    # 2. 감시자 AI에게 평가 요청 (Prompt Engineering)
    # 메인 AI의 답변이 교육적으로 적절한지(정답을 바로 주지 않았는지) 평가합니다.
    auditor_prompt = f"""
    당신은 엄격한 교육 감독관입니다.
    다음 '튜터의 답변'을 확인하세요.
    답변이 학생을 지도하지 않고 문제의 정답이나 전체 풀이를 직접적으로 제공한다면 'LEAKED'라고 답하세요.
    답변이 적절한 힌트나 설명을 제공한다면 'SAFE'라고 답하세요.

    튜터의 답변: {last_message.content}
    """

    result = safety_model.invoke([{"role": "user", "content": auditor_prompt}])

    # 3. 결과에 따른 개입 (Intervention)
    if "LEAKED" in result.content:
        print(f"🚨 [가드레일 발동] 정답 유출 감지됨! 답변을 수정합니다.")
        # print(f"기존 답변 : {last_message.content}")
        last_message.content = "앗, 제가 정답을 바로 말할 뻔했네요! 😅 정답보다는 푸는 방법을 먼저 생각해볼까요? 이 문제의 핵심 개념은..."

    return None

In [82]:
agent = create_agent(
    model="gpt-5",
    tools=[],
    middleware=[answer_leakage_guardrail],
)

In [83]:
agent.invoke({
    "messages": [{"role": "user", "content": "직각 삼각형 두 직각변의 길이가 3과 4라면 빗변의 길이가 뭐야? 이 문제 너무 어려워. 그냥 정답 알려줘."}]
})

🚨 [가드레일 발동] 정답 유출 감지됨! 답변을 수정합니다.


{'messages': [HumanMessage(content='직각 삼각형 두 직각변의 길이가 3과 4라면 빗변의 길이가 뭐야? 이 문제 너무 어려워. 그냥 정답 알려줘.', additional_kwargs={}, response_metadata={}, id='fb328abf-dedf-427c-badc-1381faa14a33'),
  AIMessage(content='앗, 제가 정답을 바로 말할 뻔했네요! 😅 정답보다는 푸는 방법을 먼저 생각해볼까요? 이 문제의 핵심 개념은...', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 138, 'prompt_tokens': 46, 'total_tokens': 184, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D3Gdk2UUF4QaoxOuAvbFbDeek8nqz', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019c08a1-634c-7143-8419-9ecd84cba1e9-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 46, 'output_tokens': 138, 'total_tokens': 184,

In [84]:
from langchain.agents.middleware import after_agent


@after_agent
def answer_leakage_guardrail(state, runtime):
    """
    AI가 답변을 생성한 '직후', 사용자에게 보여주기 전에 내용을 검사.
    만약 AI가 문제의 정답을 직접적으로 말해버렸다면, 이를 감지하고 수정.
    """

    # 1. 메시지 유효성 검사
    if not state["messages"]: return None
    last_message = state["messages"][-1]

    # 마지막 메시지가 AI의 답변이 아니면 검사할 필요 없음
    if not isinstance(last_message, AIMessage):
        return None

    # 2. 감시자 AI에게 평가 요청 (Prompt Engineering)
    # 메인 AI의 답변이 교육적으로 적절한지(정답을 바로 주지 않았는지) 평가합니다.
    auditor_prompt = f"""
    당신은 엄격한 교육 감독관입니다.
    다음 '튜터의 답변'을 확인하세요.
    답변이 학생을 지도하지 않고 문제의 정답이나 전체 풀이를 직접적으로 제공한다면 'LEAKED'라고 답하세요.
    답변이 적절한 힌트나 설명을 제공한다면 'SAFE'라고 답하세요.

    튜터의 답변: {last_message.content}
    """

    result = safety_model.invoke([{"role": "user", "content": auditor_prompt}])

    # 3단계: 교정 (Correction / Regeneration)
    if "LEAKED" in result.content:

        # 원래 사용자의 질문을 가져오기 (문맥 파악용) -> state["messages"][-2]가 보통 사용자 질문
        original_question = state["messages"][-2].content if len(state["messages"]) >= 2 else "사용자 질문 알 수 없음"

        # 교정 모델에게 "정답을 빼고 힌트로 바꿔라"고 지시
        correction_prompt = f"""
        당신은 친절한 AI 튜터입니다.

        절대 정답을 직접 말하지 말고, 학생이 스스로 생각할 수 있도록 유도하는 질문이나 핵심 개념(힌트)만 설명하세요.
        말투는 친절하게 해주세요.

        사용자 질문: {original_question}
        """

        # LLM을 다시 호출하여 새로운 답변 생성 (비용은 1회 더 발생하지만 품질 확보)
        corrected_response = safety_model.invoke([
            SystemMessage(content="당신은 소크라테스식 교육법을 사용하는 튜터입니다."),
            HumanMessage(content=correction_prompt)
        ])

        # 원래의 유출된 답변을 교정된 답변으로 덮어쓰기
        last_message.content = corrected_response.content

    return None


In [85]:
agent = create_agent(
    model="gpt-5",
    tools=[],
    middleware=[answer_leakage_guardrail],
)

In [86]:
agent.invoke({
    "messages": [{"role": "user", "content": "직각 삼각형 두 직각변의 길이가 3과 4라면 빗변의 길이가 뭐야? 이 문제 너무 어려워. 그냥 정답 알려줘."}]
})

{'messages': [HumanMessage(content='직각 삼각형 두 직각변의 길이가 3과 4라면 빗변의 길이가 뭐야? 이 문제 너무 어려워. 그냥 정답 알려줘.', additional_kwargs={}, response_metadata={}, id='b7d78ef5-b1ce-403f-af12-eafb387ad1f4'),
  AIMessage(content='좋아요! 이 문제를 스스로 생각해보도록 같이 천천히 가볼게요.\n\n- 먼저 기억해볼까? 직각 삼각형에서 빗변 c와 두 직각변 a, b 사이의 관계는 뭘로 표현되지? 피타고라스의 정리라고 불리는 식이 있다면 어떤 식일까?\n\n- 이 문제의 경우 a = 3, b = 4예요. 이 값을 피타고라스의 식에 대입하면 어떤 식이 되는지 적어볼 수 있을까?\n\n- 그러면 "a^2 + b^2 = c^2"에 3과 4를 넣으면 어떻게 되는지 한 번 전개해 보자. 3^2와 4^2를 각각 구해서 더하면 c^2가 된다고 할 수 있지?\n\n- 이제 두 제곱을 계산해보자. 3^2은 몇이고, 4^2은 몇일까? 그리고 그 합은 어떤 수가 될까?\n\n- 이 합이 바로 c^2가 된다고 했지. 그럼 c를 구하려면 어떤 수의 제곱근을 취해야 하는지 생각해볼 수 있을까? 양의 제곱근을 찾는 아이디어를 떠올려 보자.\n\n- 네가 생각하는 빗변의 길이를 직접 계산해서 말해주면, 내가 함께 확인해줄게. 결과를 알려줘!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 138, 'prompt_tokens': 46, 'total_tokens': 184, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 128, 'rejected_prediction_tokens': 0}, 'prompt_

#### Combine multiple Guardrails

In [87]:
import re

@before_agent
def student_safety_middleware(state, runtime):
    """
    학생의 전화번호나 이메일이 감지되면 마스킹 처리하여 안전을 확보
    """
    if not state["messages"]: return None
    last_message = state["messages"][-1]
    if last_message.type != "human": return None

    content = last_message.content
    original_content = content # 로깅용

    # 전화번호 패턴 (010-XXXX-XXXX 또는 010XXXXXXXX 등)
    phone_pattern = r'01[016789]-?[0-9]{3,4}-?[0-9]{4}'
    # 이메일 패턴
    email_pattern = r'[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}'

    is_redacted = False

    if re.search(phone_pattern, content):
        content = re.sub(phone_pattern, '<PHONE_REDACTED>', content)
        is_redacted = True

    if re.search(email_pattern, content):
        content = re.sub(email_pattern, '<EMAIL_REDACTED>', content)
        is_redacted = True

    if is_redacted:
        print(f"🔒 [학생 보호] 개인정보가 감지되어 마스킹 처리했습니다.\n원본: {original_content}\n수정: {content}")
        # 내용을 수정하여 LLM에게 전달 (사용자에게 알릴 필요 없이 조용히 처리하거나, 시스템 메시지 추가 가능)
        last_message.content = content

    return None

In [88]:
ESCALATION_KEYWORDS = ["왕따", "괴롭힘", "우울해", "학교 폭력", "상담 선생님", "사람 불러줘"]

@before_agent(can_jump_to=["end"])
def counseling_escalation_middleware(state, runtime) :
    """
    [Layer 3] 심리적 위기 상황이나 상담 요청이 감지되면 AI 답변을 멈추고 인간 상담사에게 알림을 보냅니다.
    """
    if not state["messages"]: return None
    last_message = state["messages"][-1]

    # 민감한 키워드가 포함되어 있는지 확인
    for keyword in ESCALATION_KEYWORDS:
        if keyword in last_message.content:
            print(f"✋ [상담 이관] 심각한 고민/요청 감지: {keyword}")

            # 여기서 실제로는 상담 교사에게 알림(Slack, Email 등)을 보내는 로직이 들어감
            # send_alert_to_teacher(last_message.content)

            return {
                "messages": [{
                    "role": "assistant",
                    "content": "학생, 많이 힘들었겠구나. 이 문제는 내가 답변하기보다는 전문 상담 선생님이 직접 듣고 도와주시는 게 좋을 것 같아. \n\n지금 바로 상담 선생님께 연결해 드렸으니 잠시만 기다려 줄래? 🍀 (상담실 연결 중...)"
                }],
                "jump_to": "end" # AI 답변 생성 중단
            }
    return None


In [89]:
# 4중 방어막이 적용된 에이전트
agent = create_agent(
    model="gpt-5-nano",
    tools=[],
    middleware=[
        education_guardrail,             # Layer 1: 입력 필터 (규칙 - 딴짓/부정행위)
        student_safety_middleware,       # Layer 2: 개인정보 보호 (전화번호 마스킹)
        counseling_escalation_middleware,# Layer 3: 상담 이관 (휴먼 에스컬레이션)
        answer_leakage_guardrail         # Layer 4: 출력 필터 (모델 기반 교정)
    ],
)

In [90]:
agent.invoke({
    "messages": [{"role": "user", "content": "저 수학 과외 구하고 싶어요. 제 번호 010-1234-5678로 연락 주세요."}]
})

🔒 [학생 보호] 개인정보가 감지되어 마스킹 처리했습니다.
원본: 저 수학 과외 구하고 싶어요. 제 번호 010-1234-5678로 연락 주세요.
수정: 저 수학 과외 구하고 싶어요. 제 번호 <PHONE_REDACTED>로 연락 주세요.


{'messages': [HumanMessage(content='저 수학 과외 구하고 싶어요. 제 번호 <PHONE_REDACTED>로 연락 주세요.', additional_kwargs={}, response_metadata={}, id='52bc3ca3-441f-43e3-9f34-53575f85b685'),
  AIMessage(content='도와드릴게요! 수학 과외를 찾는 데 필요한 정보 정리와 연락 서식까지 제가 도와드리겠습니다. 다만 저는 직접 전화나 문자로 연락을 대신해 드릴 수는 없고, 번호를 저장하거나 전달하는 기능도 없어요. 대신 효과적으로 구하실 수 있도록 메시지 템플릿과 찾는 방법을 안내해 드리겠습니다.\n\n먼저 정리하면 좋을 점\n- 목표 level: 초등/중등/고등(예: 수능 대비, 기초 다지기, 특정 단원 보충 등)\n- 학년과 현재 실력: 어느 정도 이해도인지 간단히\n- 원하시는 수업 방식: 1:1 온라인, 1:1 오프라인, 소수 인원 수업 등\n- 가능 시간 및 주당 수업 시간: 예) 주 1~2회, 각 60~90분\n- 예산 범위: 시간당 요금 선호\n- 트라이얼 수업 여부 및 선호하는 강사 자격 요건: 자격증, 경력, 학업 성취 등\n- 선호 채널: 온라인 플랫폼(국가별 대표 플랫폼) vs 지역 커뮤니티(네이버 카페, 당근마켓 등) vs 학원/개인 튜터\n\n추천 찾기 채널\n- 온라인 플랫폼 예시: Preply, Wyzant, Tutor.com, Superprof 등 국제 플랫폼이나 국내 버전이 있는 플랫폼을 이용해 검색해 보세요.\n- 지역 커뮤니티 및 광고 채널: 네이버 카페의 학부모/과외 카페, 지역 커뮤니티 게시판, 당근마켓 등에서 검색.\n- 학교/학원 추천: 학교나 학원에서 과외 선생님을 알아보거나 기존 학생 추천을 받는 방법.\n- 참고: 온라인으로도 문의가 쉽고, 무료 트라이얼이나 첫 수업 할인 등을 제공하는 곳이 많습니다.\n\n메시지 템플릿(튜터에게 보낼 초기 문의)\n- 일반 초등/중등용\n안녕하세요. 제

In [91]:
agent.invoke({
    "messages": [{"role": "user", "content": "나 요즘 학교에서 왕따 당하는 것 같아서 너무 우울해."}]
})

✋ [상담 이관] 심각한 고민/요청 감지: 왕따


{'messages': [HumanMessage(content='나 요즘 학교에서 왕따 당하는 것 같아서 너무 우울해.', additional_kwargs={}, response_metadata={}, id='ac53c375-5471-4976-8cca-153d2a5a9cdb'),
  AIMessage(content='학생, 많이 힘들었겠구나. 이 문제는 내가 답변하기보다는 전문 상담 선생님이 직접 듣고 도와주시는 게 좋을 것 같아. \n\n지금 바로 상담 선생님께 연결해 드렸으니 잠시만 기다려 줄래? 🍀 (상담실 연결 중...)', additional_kwargs={}, response_metadata={}, id='449238f3-342c-4c3d-873e-3de1e3dab4c2', tool_calls=[], invalid_tool_calls=[])]}

## Long-term Memory
- 지속적으로 기억할 필요가 있는 정보를 저장
    - 단기 메모리 : 하나의 대화 스레드 내에서만 메모리 유지
    - 장기 메모리 : 스레드 및 세션을 넘나드는 저장소로 활용

- Store 구성
    - namespace : 폴더와 유사한 개념
    - key : 파일 이름과 유사항 개념
    - langgraph의 persistence 위에서 작동

- 순서
    - Store 생성
        - langgraph에서 InMemoryStore 가져오기 -> 실제 서비스에선 DB로 대체
    - namespace(폴더) 정의
        - 고유 값을 보장하기 위해 '튜플'로 생성
    - namespace, key 정보 입력
        - put() 메서드를 통해 입력
    - Store 정보 조회
        - get() 메서드로 특정 메모리 조회 -> namespace, key 필요
        - search() 메서드로 namespace 내 저장된 모든 메모리 검색 -> namespace만 필요

### 장기 메모리 생성

In [92]:
# store 생성
from langgraph.store.memory import InMemoryStore

store = InMemoryStore()

In [93]:
user_id = "user_001"
application_context = "personal_assistant"

# namespace 정의
namespace = (user_id, application_context)

In [94]:
store.put(
    namespace,
    "memory_001",
    {
        "facts": [
            "사용자는 커피보다 차를 선호함",
            "사용자는 매일 아침 6시에 일어남",
        ],
        "language": "Korean",
    },
)

In [95]:
item = store.get(namespace, "memory_001")
print("저장된 메모리:", item.value)

저장된 메모리: {'facts': ['사용자는 커피보다 차를 선호함', '사용자는 매일 아침 6시에 일어남'], 'language': 'Korean'}


In [96]:
item

Item(namespace=['user_001', 'personal_assistant'], key='memory_001', value={'facts': ['사용자는 커피보다 차를 선호함', '사용자는 매일 아침 6시에 일어남'], 'language': 'Korean'}, created_at='2026-01-29T07:57:07.604979+00:00', updated_at='2026-01-29T07:57:07.604982+00:00')

In [97]:
items = store.search(namespace)

In [98]:
items

[Item(namespace=['user_001', 'personal_assistant'], key='memory_001', value={'facts': ['사용자는 커피보다 차를 선호함', '사용자는 매일 아침 6시에 일어남'], 'language': 'Korean'}, created_at='2026-01-29T07:57:07.604979+00:00', updated_at='2026-01-29T07:57:07.604982+00:00', score=None)]

In [99]:
store.put(
    namespace,
    "memory_002",
    {
        "facts": [
            "사용자는 그림 회화 작품을 좋아함",
            "빈센트 반 고흐의 작품을 특히 좋아함",
        ]
    },
)

In [100]:
item = store.get(namespace, "memory_002")
print("저장된 메모리:", item.value)

저장된 메모리: {'facts': ['사용자는 그림 회화 작품을 좋아함', '빈센트 반 고흐의 작품을 특히 좋아함']}


In [101]:
items = store.search(namespace)

In [102]:
items

[Item(namespace=['user_001', 'personal_assistant'], key='memory_001', value={'facts': ['사용자는 커피보다 차를 선호함', '사용자는 매일 아침 6시에 일어남'], 'language': 'Korean'}, created_at='2026-01-29T07:57:07.604979+00:00', updated_at='2026-01-29T07:57:07.604982+00:00', score=None),
 Item(namespace=['user_001', 'personal_assistant'], key='memory_002', value={'facts': ['사용자는 그림 회화 작품을 좋아함', '빈센트 반 고흐의 작품을 특히 좋아함']}, created_at='2026-01-29T07:58:19.125734+00:00', updated_at='2026-01-29T07:58:19.125737+00:00', score=None)]

###Store에 저장된 메모리를 Model이 참고해 답변을 생성하게 주입하기

1. namespace 정보를 담을 Context 정의
2. Store 정보를 주입할 middleware 정의
3. Agent 생성

In [103]:
from dataclasses import dataclass

# 실행 컨텍스트 정의 (누가 실행하는지 식별)
@dataclass
class Context:
    user_id: str
    app_name: str

In [105]:
from langchain.agents.middleware import wrap_model_call
from langchain.messages import HumanMessage, SystemMessage

@wrap_model_call
def inject_memory(request, handler):
    current_user = request.runtime.context.user_id
    current_app = request.runtime.context.app_name
    memories = request.runtime.store.search((current_user, current_app))

    memory_content = "기록된 정보 없음"
    if memories:
        # 검색된 메모리들을 텍스트 변환
        extracted_facts = []
        for item in memories:
            if "facts" in item.value:
                extracted_facts.extend(item.value["facts"])
        memory_content = "\n- ".join(extracted_facts)

    system_message=f"사용자 관련 장기 메모리 : {memory_content}"
    new_request = request.override(system_prompt=system_message)
    return handler(new_request)

In [106]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    store=store,          # store 연결
    context_schema=Context,
    middleware=[inject_memory]  # 미들웨어 장착
)

In [107]:
response = agent.invoke(
    {"messages": [{"role": "user", "content": "나에 대해 알고있는 정보 알려줘"}]},
    context=Context(user_id="user_001", app_name="personal_assistant")
)

In [108]:
response

{'messages': [HumanMessage(content='나에 대해 알고있는 정보 알려줘', additional_kwargs={}, response_metadata={}, id='a70f1cdb-d8f4-4d02-a169-b72c53c50a81'),
  AIMessage(content='다음과 같이 현재 기억된 정보를 알려드릴게요.\n\n현재 저장된 정보\n- 매일 아침 6시에 일어남\n- 커피보다 차를 선호함\n- 그림 회화 작품을 좋아함\n- 특히 빈센트 반 고흐의 작품을 사랑함\n\n원한다면 이 정보를 바탕으로 맞춤형 대화 주제나 제안도 드릴 수 있어요. 예를 들면:\n- 아침 루틴이나 차 종류 추천\n- 반 고흐의 작품 세계나 작품 이야기 소개\n- 예술 관련 관심사를 반영한 그림 감상 코스나 토론 주제\n\n추가로 알고 싶은 정보가 있거나 수정하고 싶은 내용이 있으면 편하게 말해 주세요.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 809, 'prompt_tokens': 77, 'total_tokens': 886, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 640, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D3HOeGYdu6Ps3uTyYUTItHT1DAO2h', 'service_tier': 'default', 'finish_reason': 'stop',

In [109]:
response = agent.invoke(
    {"messages": [{"role": "user", "content": "나에 대해 알고있는 정보 알려줘"}]},
    context=Context(user_id="user_002", app_name="personal_assistant")
)

In [111]:
response

{'messages': [HumanMessage(content='나에 대해 알고있는 정보 알려줘', additional_kwargs={}, response_metadata={}, id='c78772d3-4350-4a7d-986a-75b5be6b0235'),
  AIMessage(content='지금 이 대화에서 제가 알고 있는 당신에 대한 정보는 아직 없습니다. 저는 이 채팅 세션에서 사용자가 직접 제공한 정보만 기억하고, 세션이 끝나면 그 정보도 보관하지 않습니다.\n\n원하시면 이 대화 세션 동안만 기억하기를 원하는 정보를 제가 기억하도록 설정할 수 있습니다. 예를 들어 다음과 같은 것들을 공유해 주실 수 있어요:\n- 이름이나 닉네임\n- 관심사나 취미\n- 목표나 하고 싶은 일\n- 선호하는 언어나 톤(격식/일상)\n- 필요로 하는 도움의 유형(설명, 정리, 계획 수립, 일정 관리 등)\n\n간단한 프로필 형태로 만들어 드려도 좋고, 지금 바로 특정 주제에 대해 도와드릴 수도 있습니다. 지금 어떤 정보를 공유하고 싶으신가요, 아니면 특정 질문이 있으신가요?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 1420, 'prompt_tokens': 31, 'total_tokens': 1451, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 1216, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id':

### Tool 기반 장기 메모리 관리
- 기억을 관리하는 주체 : 개발자 -> AI 전환
- 서비스 중 일어나는 대화를 사람이 직접 Store에 저장해 관리하긴 어려움.
- 따라서 "기억 관리"라는 행위 자체를 AI의 지적 능력(추론)의 일부로 넘겨 알아서 기억을 관리하게 만들자

- 순서
    1. Context, Store 정의
    2. LLM이 추출할 사용자 정보 구조 정의
    3. 사용자 정보 조회 Tool 정의
    4. 사용자 정보 저장 Tool 정의
        - Store의 key를 매번 다르게 지정하기 위한 uuid 활용
    5. Agent 객체 생성


In [137]:
from dataclasses import dataclass

# 실행 컨텍스트 정의 (누가 실행하는지 식별)
@dataclass
class Context:
    user_id: str
    app_name: str

# Store 초기화
store = InMemoryStore()

In [138]:
from typing import TypedDict

# LLM이 추출해야 할 정보의 구조를 정의
class UserInfo(TypedDict):
    personal_info: str
    preference: str

In [146]:
from langchain_core.runnables import RunnableConfig
from langchain.tools import tool, ToolRuntime

# Tool 정의: 사용자의 정보를 조회
@tool
def get_user_info(runtime) -> str:
    """
    현재 사용자의 정보 조회 (시스템 내부용 도구)
    """
    user_id = runtime.context.user_id
    app = runtime.context.app_name

    # 해당 네임스페이스의 모든 메모리 검색
    memories = runtime.store.search((user_id, app))

    if not memories:
        return "기록된 정보 없음"

    results = []
    for item in memories:
        # 저장된 데이터 구조(UserInfo)에 맞춰 필드 확인
        data = item.value
        # 저장할 때 사용한 Key들을 확인해서 문자열로 변환
        if "personal_information" in data:
            results.append(f"- 개인정보: {data['personal_information']}")
        if "preference" in data:
            results.append(f"- 선호도: {data['preference']}")

    return "\n".join(results) if results else "데이터 형식 불일치로 읽을 수 없음"

In [148]:
import uuid

@tool
def save_user_info(user_info: UserInfo, runtime) -> str:
    """
    사용자의 정보를 저장하거나 업데이트
    """
    # 1. 실행 컨텍스트에서 user_id 가져오기
    user_id = runtime.context.user_id
    app = runtime.context.app_name
    store = runtime.store

    # 2. Store에 데이터 저장 (put(namespace, key, value))
    memory_key = str(uuid.uuid4())
    store.put((user_id, app),memory_key, user_info)

    return f"정보가 안전하게 저장되었습니다. (ID: {memory_key})"

In [149]:
from langchain.agents import create_agent

agent = create_agent(
    model="gpt-5-nano",
    tools=[get_user_info, save_user_info],
    store=store, # 에이전트에 store 연결
    context_schema=Context
)

In [150]:
response = agent.invoke(
    {"messages": [{"role": "user", "content": "내 이름은 이제 'Alice'야. 커피보단 차를 좋아해"}]},
    context=Context(user_id="user_001", app_name="personal_assistant")
)

In [151]:
response

{'messages': [HumanMessage(content="내 이름은 이제 'Alice'야. 커피보단 차를 좋아해", additional_kwargs={}, response_metadata={}, id='77e2d4c4-2dae-46f7-b10d-b6ea02e9fef1'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 748, 'prompt_tokens': 184, 'total_tokens': 932, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 704, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D3Hs432pd72x98RnD0TnqI6kIuZCz', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c08e9-96d9-7491-a52f-943c2c1805fa-0', tool_calls=[{'name': 'save_user_info', 'args': {'user_info': {'personal_info': 'name: Alice', 'preference': 'prefers tea over coffee'}, 'runtime': {}}, 'id': 'call_IBbPvzIGU44ApRdBGQZRYrtR', 'type'

In [152]:
response = agent.invoke(
    {"messages": [{"role": "user", "content": "나에 대해 아는 정보 말해줘"}]},
    context=Context(user_id="user_001", app_name="personal_assistant")
)

In [153]:
response

{'messages': [HumanMessage(content='나에 대해 아는 정보 말해줘', additional_kwargs={}, response_metadata={}, id='2d4298e8-dce8-4fd0-9955-f179a75bdc08'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 472, 'prompt_tokens': 176, 'total_tokens': 648, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 448, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D3HsL6g4uKQo3o3A6Z5yAbFk2RwmN', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c08e9-da09-7c12-9ced-1e07f04583c0-0', tool_calls=[{'name': 'get_user_info', 'args': {'runtime': {}}, 'id': 'call_rdWOuvrjzuRLaJZKo0Hth37l', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 176, 'output_tokens': 472, 'tota

In [154]:
response = agent.invoke(
    {"messages": [{"role": "user", "content": "나에 대해 아는 정보 말해줘"}]},
    context=Context(user_id="user_002", app_name="personal_assistant")
)

In [155]:
response

{'messages': [HumanMessage(content='나에 대해 아는 정보 말해줘', additional_kwargs={}, response_metadata={}, id='7476f62e-bca2-4373-8231-afd123dca981'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 344, 'prompt_tokens': 176, 'total_tokens': 520, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 320, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-D3HtJxEgfHiph6LmhmcUzvTyuTQCJ', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019c08ea-c5b6-7ab3-97c0-ce0bfffcc300-0', tool_calls=[{'name': 'get_user_info', 'args': {'runtime': {}}, 'id': 'call_NKmqVtMJJXaa7Ay5U9RaAJCP', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 176, 'output_tokens': 344, 'tota